Google Colab Specific loading

In [1]:
!rm -r FNB_grading/
!rm preprocessing.py
!git clone https://git@github.com/justinwhatley/FNB_grading.git
!mv FNB_grading/preprocessing.py preprocessing.py

rm: cannot remove 'FNB_grading/': No such file or directory
Cloning into 'FNB_grading'...
remote: Counting objects: 59, done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 59 (delta 23), reused 53 (delta 17), pack-reused 0
Unpacking objects: 100% (59/59), done.
Checking connectivity... done.


Loading Dataset

In [ ]:
# Installing dependencies
!pip install PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from keras

In [ ]:
# Authorizing Google SDK
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
# Downloads raw data from Google Drive and extracts it
!rm -rf FNAB_raw
download = drive.CreateFile({'id': '10aVomUhZyaMiJuqfSBEzcwWVKkzE7hBN'})
download.GetContentFile('FNAB_raw.tar.gz')
!tar -xzf FNAB_raw.tar.gz

In [ ]:
# Downloads preprocessed data from Google Drive and extracts it
!rm -rf FNAB_preprocessed
download = drive.CreateFile({'id': '1Amc6q-XfcErdimjPxtuAGo_jpDay0aqw'})
download.GetContentFile('FNAB.preprocessed.tar.gz')
!tar -xzf FNAB.preprocessed.tar.gz;
!ls

In [1]:
# Sets the path based on the environment being used

def get_path(computer_str):
    """
    Returns the correct path based on where the program is run
    """

    mac_data_path = '/Users/justinwhatley/Dropbox/FevensLab'
    linux_data_path = '/home/justin/Dropbox/FevensLab'
    
    if computer_str.lower() == 'mac':
        return os.path.join(mac_data_path)

    elif computer_str.lower() == 'linux': 
        return os.path.join(linux_data_path)

    elif computer_str.lower() == 'colab':
        # Google colab path
        return os.path.join('')

    else: 
        print('Incorrect base path option')
        exit(0)

Separates dataset into folds

In [2]:
import preprocessing
import os

In [54]:
!rm -rf training_validation_dataset/

base_directory = get_path('linux')
raw_data_directory_path = os.path.join(base_directory, 'FNAB_raw')
preprocessed_directory_path = os.path.join(base_directory, 'FNAB_preprocessed')
training_validation_path = os.path.join(base_directory, 'training_validation_dataset')

class_keyword_1 = 'MG2'
class_keyword_2 = 'MG3'
classes_list = [class_keyword_1, class_keyword_2]

# Prepares data
height, width = 224, 224

files_list_by_class = preprocessing.get_raw_file_list(raw_data_directory_path, classes_list)
preprocessing.create_preprocessed_directory(classes_list, files_list_by_class, preprocessed_directory_path, height, width, overwrite_previous_preprocessed_data = False)

# Gets patch file data
patched_class_file_list = preprocessing.get_data_by_class(os.path.join(preprocessed_directory_path, 'patched_data'), classes_list)
original_class_file_list = preprocessing.get_data_by_class(os.path.join(preprocessed_directory_path, 'original_data'), classes_list)

# Separate data into k-folds
number_of_folds = 5
files_per_fold =  [4000, 700]
patched_files_in_folds = preprocessing.separate_into_k_folds(number_of_folds, patched_class_file_list, files_per_fold)
# print(patched_files_in_folds)

files_per_fold =  [90, 20]
original_files_in_folds = preprocessing.separate_into_k_folds(number_of_folds, original_class_file_list, files_per_fold)
# print(original_files_in_folds)

# Iterates through folds

for i in range(number_of_folds):   
    validation_fold = i
    
    # Removes previous training and validation directories
    preprocessing.remove_dir(training_validation_path)
                                       
    # Selects a validation and training set on the original data
    preprocessing.assign_folds_to_training_and_validation(preprocessed_directory_path, training_validation_path, classes_list, original_files_in_folds, validation_fold, type = 'original_data')

    # Selects a validation and training set on the patched data
#     preprocessing.assign_folds_to_training_and_validation(preprocessed_directory_path, training_validation_path, classes_list, patched_files_in_folds, validation_fold, type = 'patched_data')

    # TODO call training and validation from here
    print('Finished! ')
    break


Loading raw data from path: /home/justin/Dropbox/FevensLab/FNAB_raw
82
130
Bin size: 4596
Bin size: 4314
Bin size: 4320
Bin size: 4374
Bin size: 4500
Bin size: 919
Bin size: 1015
Bin size: 761
Bin size: 769
Bin size: 1012
82
24
Bin size: 92
Bin size: 92
Bin size: 92
Bin size: 95
Bin size: 97
Bin size: 20
Bin size: 26
Bin size: 20
Bin size: 22
Bin size: 18
Removing directory: /home/justin/Dropbox/FevensLab/training_validation_dataset
Taking files from input directory: /home/justin/Dropbox/FevensLab/FNAB_preprocessed/original_data/MG2
Storing them in output directory: /home/justin/Dropbox/FevensLab/training_validation_dataset/original_data/training/MG2
Taking files from input directory: /home/justin/Dropbox/FevensLab/FNAB_preprocessed/original_data/MG2
Storing them in output directory: /home/justin/Dropbox/FevensLab/training_validation_dataset/original_data/validation/MG2
Taking files from input directory: /home/justin/Dropbox/FevensLab/FNAB_preprocessed/original_data/MG3
Storing them in

Neural Network

In [68]:
import numpy as np
import keras
import os.path as path
from keras.preprocessing.image import ImageDataGenerator

from keras import optimizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization 
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [7]:
# TODO figure out why bins are artificially double the size
ls -l FNAB_preprocessed/original_data/MG2/ | egrep -c '^-'

SyntaxError: invalid syntax (<ipython-input-7-a28e9f191fcb>, line 2)

In [ ]:
# TODO figure out why there are incorrect (small) numbers of files in the different training and validation sets
ls -l training_validation_dataset/patched_data/training/MG3 | egrep -c '^-'

In [84]:
""" Hyperparameters """


# Dimensions images will be resized to for processing
img_width, img_height = 224, 224

class_list = ['MG2', 'MG3']

epochs = 50
batch_size = 32

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [87]:
""" Calculates the number of training and validation samples"""
# dataset_name = 'training_validation_dataset/patched_data'

# Loads the unaltered images that have been place in training and validation datasets 
# dataset_name = os.path.join(training_validation_path, 'patched_data')
dataset_name = os.path.join(training_validation_path, 'original_data')
train_data_dir = path.join(dataset_name, 'training')
validation_data_dir = path.join(dataset_name, 'validation')

# Calculate number of files in directory 
import os
import glob

def get_file_list(path_regex):
    file_list = []
    for filename in glob.glob(path_regex): 
        file_list.append(filename)
    return file_list
  
# Loads jpg files using regex
path_regex = path.join(train_data_dir + '/*/*.jpg')
nb_train_samples = len(get_file_list(path_regex))
print('Number of training samples: '+ str(nb_train_samples))

path_regex = path.join(validation_data_dir + '/*/*.jpg')
nb_validation_samples = len(get_file_list(path_regex))
print('Number of validation samples: '+ str(nb_validation_samples))


Number of training samples: 438
Number of validation samples: 110


In [88]:
""" Defines the model """

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# model.summary()

In [80]:
# Laya's first CNN

model = Sequential()

model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), input_shape=input_shape))
model.add(Activation(Dense(128, activation='relu')))

model.add(Conv2D(128, kernel_size=(3, 3), strides=(1, 1)))
model.add(Activation(Dense(128, activation='relu')))

model.add(Dropout(0.5))
model.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1)))

model.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1)))
model.add(Activation(Dense(64, activation='relu')))

model.add(Conv2D(64, kernel_size=(3, 3), strides=(1, 1)))
model.add(Activation(Dense(64, activation='relu')))

model.add(Dropout(0.5))
model.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1)))

model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1)))
model.add(Activation(Dense(32, activation='relu')))

model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1)))
model.add(Activation(Dense(32, activation='relu')))

model.add(Dropout(0.5))
model.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1)))

model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
metrics=['accuracy'])

/home/justin/anaconda3/lib/python3.6/site-packages/keras/activations.py:197: UserWarning: Do not pass a layer instance (such as Dense) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [73]:
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))
        
history = AccuracyHistory()

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

NameError: name 'x_train' is not defined

In [89]:
""" Training and validation for regular network """

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps= nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Found 438 images belonging to 2 classes.
Found 110 images belonging to 2 classes.
Epoch 1/50


ValueError: Error when checking target: expected activation_91 to have shape (2,) but got array with shape (1,)

In [ ]:
ls training_validation_dataset/patched_data/training/MG3/

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model CNN:  accuracy history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
# plt.grid(b=False)
# plt.savefig('accuracy.png', bbox_inches='tight', dpi=300)
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model CNN:  loss history')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
# plt.grid(b=False, )
# plt.savefig('loss.png', bbox_inches='tight', dpi=300)
plt.show()

# from google.colab import files
# files.download('loss.png')
# files.download('accuracy.png')